In [1]:
 %reload_ext autoreload


In [2]:
%autoreload

from model.base import Base
from model.head import Head
from model.finetune_model import Fmodel
import torch
import h5py
import numpy as np
import os
from utils import build_h5,extract_features, get_model_from_ckpt,train,eval,lr_scheduler, makedirs,save_model,logging
from torch.optim import Adam
import gc
import re
from dataloader import get_dataloader

hard_dp = False

if hard_dp:
    dropout = [0.5,0.5,0.75]
else:
    dropout = [0.25,0.25,0.5]

base_model_type = 'inceptionv3'
num_level = 11
feature_type = 'wide'
head_type = 'multi_3FC'

if feature_type == 'wide':
    head_type = 'pool_3FC'

resize = True
augment = True
finetune = True
root = './experiment/'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
folder = makedirs(root,base_model_type,num_level,feature_type,head_type,resize,augment,hard_dp)

splits = ['train', 'val', 'test']
h5_paths = [os.path.join(folder,split + '_fea.h5') for split in splits]

train_batch_size = 1

imgloaders = get_dataloader(data_type='img',resize=resize,finetune = finetune,augment=augment,batch_size = train_batch_size, head_type = head_type)

ImportError: cannot import name 'flip' from 'torchvision.transforms.functional' (C:\Users\User\Anaconda3\envs\khiem\lib\site-packages\torchvision\transforms\functional.py)

In [4]:
train_batch_size = 2

imgloaders = get_dataloader(data_type='img',resize=resize,augment=augment,batch_size = train_batch_size, head_type = head_type)
fealoaders = get_dataloader(data_type='fea', head_type = head_type, h5_paths = h5_paths, batch_size = train_batch_size)

In [6]:
continue_training = False

head = Head(head_type,fealoaders[0].dataset.channel_size,dropout)
fmodel = Fmodel(base_model_type,feature_type, head, num_level)

if continue_training:
    fmodel,latest_ep = get_model_from_ckpt(folder,fmodel, model_type = 'fmodel',metric= 'ep')
else:
    fmodel.head = get_model_from_ckpt(folder, fmodel.head,model_type = 'head', metric = 'plcc')
    latest_ep = 0


In [8]:
lr = 1e-4
epochs = 40
patience = 5
mini_iter = 2

fmodel.to(device)
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(fmodel.parameters(),lr = lr)

prev_srcc = 0
prev_plcc = 0
patience_cnt = 0

for epoch in range(epochs): 
    epoch = latest_ep+epoch
    fmodel = train(fmodel,imgloaders[0],device,loss_fn,optimizer,mini_iter,epoch)
    plcc, srcc, acc, val_loss = eval(fmodel,imgloaders[1],loss_fn,device)
    logging(epoch, val_loss,acc,plcc,srcc)

    if (plcc<prev_plcc)&(srcc<prev_srcc):
        patience_cnt+=1
    else:
        patience_cnt = 0
        print('Saving ckpt')
        save_model(fmodel,folder,epoch,plcc,srcc,acc,'fmodel')
        prev_plcc = plcc
        prev_srcc = srcc

    optimizer = lr_scheduler(optimizer, epoch, lr, patience_cnt)

    if patience_cnt == patience:
        if lr == 1e-6:
            print('CONVERGE')
            break
        else:
            patience_cnt = 0
    



Epoch 0 batch0 train_loss: 0.1338847577571869
Epoch 0: val loss-0.2910360489040613 val acc-[0.20781546], srcc-[0.20672734], plcc-[0.74067595]
Saving ckpt


KeyboardInterrupt: 

In [55]:
fmodel = get_model_from_ckpt(ckpt_folder,'fmodel','srcc')
plcc, srcc, acc, val_loss = eval(fmodel,imgloaders[2],loss_fn,device)
print(f'Test loss-{val_loss}')
print(f'Test acc-{acc}, srcc-{srcc}, plcc-{plcc}')  


Test loss-0.23576870788820087
Test acc-[0.84012202], srcc-[0.28929608], plcc-[0.41180513]
